In [ ]:
from vizdoom import *
import numpy as np
import random

#initialize
qvalue = np.zeros([16,4])
evalue = np.zeros([16,4])

#slippery
env = gym.make('FrozenLake-v0')

#non-slippery
# from gym.envs.registration import register
# register(
#     id='FrozenLakeNotSlippery-v0',
#     entry_point='gym.envs.toy_text:FrozenLakeEnv',
#     kwargs={'map_name' : '4x4', 'is_slippery': False},
#     max_episode_steps=100,
#     reward_threshold=0.8196, # optimum = .8196, changing this seems have no influence
# )
# env = gym.make("FrozenLakeNotSlippery-v0")

#hyperparameters
alphaLearningRate = 0.3
faresightLambda = 0.95
decayRate = 0.0
decay_rate_of_epsilon = 0.001
max_epsilon = 1.0
min_epsilon = 0.00001 

#record score
score = []

def choose_action(state, epsilon):
    if random.random() >= epsilon:
        return np.argmax(qvalue[state])
    else:
        return random.randrange(4)

def SARSAcontrol():
    epsilon = 1.0
    for i in range(20000):
        previousState = env.reset()
        previousAction = choose_action(previousState, epsilon)
        evalue = np.zeros([16,4])
        score = []
        done = False
        while not done:
            steps = 0
            currentState, reward, done, info = env.step(previousAction)
            steps += 1
            currentAction = choose_action(currentState, epsilon)
            error = reward + faresightLambda * qvalue[currentState][currentAction] \
                - qvalue[previousState][previousAction]
            evalue[previousState][previousAction] += 1
            
            nonZeroes = np.transpose(np.nonzero(evalue))
            for nonZeroe in nonZeroes:
                row = nonZeroe[0]
                column = nonZeroe[1]
                qvalue[row][column] += alphaLearningRate * evalue[row][column] * error
            
            evalue *= decayRate * faresightLambda
            
            
            previousState = currentState
            previousAction = currentAction
        epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate_of_epsilon*i)
                
    print("training end")
    print(qvalue)
    
def evaluation():
    score = []
    done = False
    for i in range(10000):
        observation = env.reset()
        while True:
            action = choose_action(observation, 0)
            observation, reward, done, info = env.step(action)
            if done:
                score.append(reward)
                break
    
    print("accuracy: {}".format(np.mean(score)))
